In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('../data/res_mapping.csv', nrows=1000)
data['res_dt']=pd.to_datetime(data['res_dt'], format="%d.%m.%y %H:%M")
data['dep_soll']=pd.to_datetime(data['dep_soll'], format="%d.%m.%y %H:%M")
data['diff']=abs((data['res_dt']-data['dep_soll']).dt.total_seconds())
data

In [ ]:
tokens = list(set(np.hstack([data['bp_from'], data['bp_to']])))

In [ ]:
for index, row in data.iterrows():
    data.at[index, 'bp_from_token'] = tokens.index(row['bp_from']) / len(tokens)
    data.at[index, 'bp_to_token'] = tokens.index(row['bp_to']) / len(tokens)

In [ ]:
data

In [ ]:
data.to_csv('../data/output.csv', columns = ['diff', 'bp_from_token', 'bp_to_token'], index=False)